In [1]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell (on MacOS press 'Cmd+Enter')

# 2. run this cell
import os
import sys
from pathlib import Path

# add path to domblar development code
sys.path.insert(0, str(Path(os.getcwd()).parent))

rep = 3
from domblar.domblar import Domblar
d = Domblar(
    synth_count=3 * rep + 2,
    context='dexed'
)

/Users/gexahedron/devel/domblar


In [2]:
# tintinnabuli example (from Fratres)
# based on https://www.youtube.com/watch?v=XbykaYwVO0w

# TODO:
# replace rep, muls & amps with something smarter and less error-prone
# humanize tempo
# humanize drone (add some variation to dynamics or to timbre)
# generalize to other scales (e. g., 22edo and choose some porcupine MODMOS)
# create useful operations for live-coding/composition
# (maybe) add timbre modulations
# (maybe) calculate t-scales with some algorithms

import time

from domblar.transformations import transpose
from domblar.edo import Scale, get_freq
from domblar.music_math import freq_to_midi


def set_synths():
    for i in range(3 * rep):
        d.set_synth(i, 'strings10')
    d.set_synth(3 * rep, 'strings5')
    d.set_synth(3 * rep + 1, 'claves')
set_synths()

edo = 12
# TODO: replace this list(range(edo)) with something more meaningful
scale = list(range(edo))

# t and m scales
m_scale  = Scale([ 4, 5, 7, 9, 10, 13, 14], edo)  # D harm min / A Phrygian dominant
t1_scale = Scale([-3, 0, 0, 0,  0,  4,  4], edo)  # Am chord; A natural minor (implied)
t2_scale = Scale([-3, -3, 0, 0, 4,  4,  9], edo)  # also Am chord
t_scales = [t1_scale, t2_scale]


# start drone
note = 9 - edo * 3
freq = get_freq(note, scale, edo)
drone_idx = 3 * rep
midi_note, midi_bend = freq_to_midi(freq, synth_idx=drone_idx)
channel = 0
dur = 0
amp = 0.4
state = 1
data_start_drone = [drone_idx, midi_note, midi_bend, dur, int(amp * 127), channel, state]
state = 2
data_end_drone = [drone_idx, midi_note, midi_bend, dur, int(amp * 127), channel, state]
d.client.send_msg('/play', data_start_drone, timetag=time.time())

def build_chord(idx, m_scale, t_scale):
    return transpose([
        m_scale[idx - 2],
        transpose(t_scale[idx], edo),
        transpose(m_scale[idx], edo)
        ], edo)

chords = []
muls = []
amps = []

def add_claves(amp):
    # rhythm: 'x.xx..x.xx..'
    for i in range(2):
        for j in range(1, 3):
            chords.extend([['.', '.', '.', 0]] * j)
            muls.extend([1] * j)
            amps.extend([amp * 0.5] * j)
            chords.extend([['.']] * j)
            muls.extend([1] * j)
            amps.extend([0] * j)

rounds = 8
for r in range(rounds + 1):
    cur_amp = 1.5 - 1.5 * abs(r - 3) / 6

    add_claves(cur_amp)
    if r == rounds:
        break

    for k in range(2):
        direction = -1 if k == 0 else 1
        for i in range(1, 4):
            chords.append(build_chord(0 - 2 * r, m_scale, t_scales[k]))
            muls.append(2)
            amps.append(cur_amp)
            for j in range(1, i + 1):
                chords.append(build_chord(j * direction - 2 * r, m_scale, t_scales[k]))
                muls.append(1)
                amps.append(cur_amp)
            for j in range(i):
                chords.append(build_chord((j - i) * direction - 2 * r, m_scale, t_scales[k]))
                muls.append(1)
                amps.append(cur_amp)
            chords.append(build_chord(0 - 2 * r, m_scale, t_scales[k]))
            muls.append(2)
            amps.append(cur_amp)
            chords.append('.')
            muls.append(1)
            amps.append(cur_amp)

dur = 0.9
d.play(chords, scale, edo,
       synth_idx=[0, 3, 6, 10], rep=[rep, rep, rep, 1],
       dur=dur, sus=dur * 0.9,
       muls=muls, amps=amps)

# end drone
d.client.send_msg('/play', data_end_drone, timetag=time.time())

KeyboardInterrupt: 

In [5]:
finetuning = 2

try:
    finetuning
except Exception as e:
    d.open_editor(0)
    finetuning = 1
else:
    if finetuning == 0:
        d.open_editor(0)
    elif finetuning == 1:
        # transfer instrument to all synths
        d.save_preset(0)
        import time
        time.sleep(0.5)
        for i in range(3 * rep):
            d.load_preset(i)
    else:
        d.print_params(0)

In [9]:
# cleanup
d.stop_server()